In [ ]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd
import json
import zipfile
import os

# ---------------------------
# Load and Prepare Data
# ---------------------------
# Load province polygons geoJSON
with open('geoBoundaries-CAN-ADM1_simplified.geojson') as f:
    geojson_data = json.load(f)

gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
gdf = gdf.rename(columns={"shapeName": "Province"})
gdf.set_crs(epsg=4326, inplace=True)

# Define notable places per province (as lists)
province_to_places = {
    "Alberta": ["Banff NP", "Jasper NP", "Calgary Tower", "Lake Louise", "West Edmonton Mall"],
    "British Columbia": ["Stanley Park", "Butchart Gardens", "Whistler", "Capilano Bridge", "Pacific Rim NP"],
    "Manitoba": ["The Forks", "Riding Mountain NP", "Assiniboine Zoo", "Museum for Human Rights", "FortWhyte Alive"],
    "New Brunswick": ["Bay of Fundy", "Hopewell Rocks", "Fundy NP", "Reversing Falls", "Kings Landing"],
    "Newfoundland and Labrador": ["Gros Morne NP", "Signal Hill", "L'Anse aux Meadows", "Cape Spear", "Bonavista"],
    "Nova Scotia": ["Peggy's Cove", "Cabot Trail", "Halifax Citadel", "Lunenburg", "Kejimkujik NP"],
    "Ontario": ["CN Tower", "Niagara Falls", "Algonquin Park", "Parliament Hill", "Royal Ontario Museum"],
    "Prince Edward Island": ["Green Gables", "Cavindish Beach", "Confederation Trail", "PEI NP", "Point Prim Lighthouse"],
    "Quebec": ["Old Quebec", "Mont-Tremblant", "Montmorency Falls", "Quebec City", "Sainte-Anne-de-Beaupré"],
    "Saskatchewan": ["Forestry Zoo", "Wanuskewin", "Prince Albert NP", "Wascana Centre", "RCMP Heritage Centre"],
    "Northwest Territories": ["Nahanni NP", "Great Slave Lake", "Virginia Falls", "Yellowknife", "Wood Buffalo NP"],
    "Nunavut": ["Auyuittuq NP", "Sylvia Grinnell Park", "Qaummaarviit Park", "Iqaluit", "Sirmilik NP"],
    "Yukon": ["Kluane NP", "Miles Canyon", "SS Klondike", "Whitehorse", "Tombstone Park"]
}

gdf["Notable Places"] = gdf["Province"].map(lambda prov: ", ".join(province_to_places[prov]))

# Load points-of-interest
points_gdf = gpd.read_file("hotosm_can_points_of_interest_points_geojson.geojson")
points_gdf.set_crs(epsg=4326, inplace=True)
points_gdf = points_gdf.to_crs(gdf.crs)

# Precompute notable places - FIXED HERE: using unary_union instead of union_all
filtered_rows = []
for prov, places in province_to_places.items():
    province_poly = gdf[gdf["Province"] == prov].geometry.unary_union
    for place in places:
        matches = points_gdf[points_gdf["name"].str.contains(place, case=False, na=False)]
        for _, row in matches.iterrows():
            if row.geometry.within(province_poly):
                filtered_rows.append({
                    "Province": prov,
                    "Place": place,
                    "lat": row.geometry.y,
                    "lon": row.geometry.x
                })

notable_df = pd.DataFrame(filtered_rows)
notable_df["marker_id"] = notable_df.apply(lambda row: f"{row['Province']}_{row['Place']}", axis=1)

# ---------------------------
# Initialize Dash App
# ---------------------------
app = dash.Dash(__name__)
server = app.server  # Required for Binder

app.layout = html.Div([
    html.H1("Canada Provinces with Notable Places"),
    dcc.Dropdown(
        id='province-dropdown',
        options=[{'label': prov, 'value': prov} for prov in sorted(gdf['Province'].unique())],
        multi=True,
        placeholder="Select Provinces to highlight"
    ),
    dcc.Store(id='clicked-markers', data=[]),
    dcc.Graph(id='choropleth-map')
])

# [Rest of your callbacks remain the same]

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=8052)

C:\Users\faiaz\AppData\Local\Temp\ipykernel_5912\2006226598.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  province_poly = gdf[gdf["Province"] == prov].geometry.unary_union
C:\Users\faiaz\AppData\Local\Temp\ipykernel_5912\2006226598.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  province_poly = gdf[gdf["Province"] == prov].geometry.unary_union
C:\Users\faiaz\AppData\Local\Temp\ipykernel_5912\2006226598.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  province_poly = gdf[gdf["Province"] == prov].geometry.unary_union
C:\Users\faiaz\AppData\Local\Temp\ipykernel_5912\2006226598.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  province_poly = gdf[gdf["Province"] == prov].geometry.unary_union
C:\Users\faiaz\AppData\Local\Temp\ipykernel_5912\2006226